In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
os.chdir("../..")


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import json
import xsim
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output


In [9]:
from qml.db import dpo as xdpo
from qml.db import target as xtarget
from qml.db.ml import MLDatasetGenerator
from qml.model.gate import Gateset
from qml.model.unit import UnitManager, Unit
from qml.model.encoding import EncodingUnitManager
from qml.model.model import Model
from qml.optimizer import dpo as xdpopt
from qml.optimizer import evaluator as xeval
from qml.tools.random import XRandomGenerator
from qml.tools.sampler import CandidateSampler
from qml import optimizer as xoptim


## Config

In [12]:
# circuit
num_qubits = 2
num_gates = 3

# dataset
db_filename = "dpo_databsae.txt"
batch_size = 10
dim_wavelet = 4
## validation
validation_db_size = 10
dim_order = 4

# qml
num_train_step = 30

# model
dim_hiddens = [32, 32]

# training
lr = 1e-3
max_epoch = 1000
cpo = True


## Data

### QML training dataset

In [8]:
dataset = xdpo.DPODataset(db_filename, num_qubits, dim_wavelet)
loader = xdpo.DPODataLoader(dataset, num_qubits, 1, dim_wavelet)


### Validation dataset

In [16]:
tfun = xtarget.PolynominalTargetFunctionGenerator(dim_order)
tgen = MLDatasetGenerator(tfun)
validation_datasets = [
    tgen.generate(validation_db_size)
    for _ in range(5)
]


## Training